In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
 

In [13]:
from huggingface_hub import login
login(token=os.getenv("HF_TOKEN"))

/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/LLM-Fine-Tunning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [16]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('timdettmers/openassistant-guanaco')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

Repo card metadata block was not found. Setting CardData to empty.
Map: 100%|██████████| 1000/1000 [00:00<00:00, 37789.93 examples/s]


In [17]:
transformed_dataset.push_to_hub("guanaco-dataset-llama2-1k")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.81s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/msaifee/guanaco-dataset-llama2-1k/commit/13368ef54cbce3ab331239f0e1c9afc39f2b1ab4', commit_message='Upload dataset', commit_description='', oid='13368ef54cbce3ab331239f0e1c9afc39f2b1ab4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/msaifee/guanaco-dataset-llama2-1k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='msaifee/guanaco-dataset-llama2-1k'), pr_revision=None, pr_num=None)